# Training a Transformer Encoder with MLM Objective on BERT-style tokens / wikitext dataset

### 1. Setup (retrieve data, look at `mask_dataset_for_mlm` for important MLM preprocessing steps)

In [1]:
from modules.encoder import EncoderModel
from preprocess.mlm_preprocess import get_dataset_example, mask_dataset_for_mlm

input_ids, tokenizer = get_dataset_example()
mlm_input_ids, mlm_labels = mask_dataset_for_mlm(input_ids, tokenizer=tokenizer)

/Users/rishubtamirisa/miniforge3/envs/LM-impl/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset wikitext (/Users/rishubtamirisa/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 237.08it/s]
Loading cached processed dataset at /Users/rishubtamirisa/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-f23794ddde0ffc01.arrow
Loading cached processed dataset at /Users/rishubtamirisa/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-aeb8e5d15c439c91.arrow
100%|██████████| 4/4 [00:00<00:00, 19.74ba/s]


### 2. Define the model

In [2]:
import torch
from modules.encoder import EncoderModel, Encoder

vocab_size = tokenizer.vocab_size
embed_dim = 512
model_dim = 512
n_layers = 6
num_heads = 8
encoder = EncoderModel(vocab_size=vocab_size, embed_dim=embed_dim, model_dim=model_dim, n_layers=n_layers, num_heads=num_heads)
encoder.to("cpu")


EncoderModel(
  (embedding): Embedding(30522, 512)
  (pos_en): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dropout): Dropout(p=0.1, inplace=False)
  (encoder): Encoder(
    (encoder_layers): ModuleList(
      (0): EncoderBlock(
        (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (multi_head_attention): MultiHeadAttention(
          (qkv_weights_list): ModuleList(
            (0): ModuleList(
              (0): Linear(in_features=512, out_features=64, bias=True)
              (1): Linear(in_features=512, out_features=64, bias=True)
              (2): Linear(in_features=512, out_features=64, bias=True)
            )
            (1): ModuleList(
              (0): Linear(in_features=512, out_features=64, bias=True)
              (1): Linear(in_features=512, out_features=64, bias=True)
              (2): Linear(in_features=512, out_features=64, bias=True)
            )
            (2): ModuleList(
              (0): Linear(in_fea

### 3. Prepare DataLoaders

In [3]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset( mlm_input_ids, mlm_labels )
loader = DataLoader(dataset, batch_size = 32, shuffle=True)

### 4. Train

In [4]:
from tqdm import tqdm
import wandb

def train_mlm(epochs, model, tokenizer, loader, optimizer=torch.optim.Adam, device=torch.device('cpu')):
    wandb.init(
        # set the wandb project where this run will be logged
        project="transformer-mlm",
        # track hyperparameters and run metadata
        config={
        "learning_rate": optimizer.defaults['lr'],
        "architecture": "Transformer",
        "dataset": "wikitext-2",
        "epochs": epochs,
        }
    )
    criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)
    model.train()
    model.to(device)
    with tqdm(total=epochs) as pbar:
        for epoch in range(epochs):
            cur_batch = 0
            total_batches = len(loader) 
            for batch in loader:
                input_ids, labels = batch
                input_ids = input_ids.to(device, dtype=torch.int64)
                labels = labels.to(device, dtype=torch.int64)
                output = model(input_ids)
                loss = criterion(output.view(-1, vocab_size), labels.view(-1))
                wandb.log({"train_loss": loss.item()}, step=cur_batch + (epoch * total_batches))  # Log the loss
                loss.backward()
                optimizer.step()
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.zero_grad()
                cur_batch += 1
                pbar.set_postfix(**{f"Epoch {epoch} / batch: ": f"{cur_batch} / {total_batches}", "loss:": loss.item()})
        
        checkpoint = {'vocab_size': vocab_size,
                      'embed_dim': embed_dim,
                      'model_dim': model_dim,
                      'n_layers': n_layers,
                      'num_heads': num_heads,
                      'state_dict': model.state_dict()}
        torch.save(checkpoint, f'model_checkpoints/checkpoint_E{epoch}.pth')
        wandb.save(f'model_checkpoints/checkpoint_E{epoch}.pth')  # Save the model checkpoint to wandb

train_mlm(epochs=4, 
          tokenizer=tokenizer, 
          model=encoder, 
          loader=loader, 
          optimizer=torch.optim.Adam(encoder.parameters(), lr=5e-5)
          )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

wandb: Currently logged in as: rishubtamirisa. Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/4 [08:35<?, ?it/s, Epoch 0 / batch: =125 / 743, loss:=7.39]


KeyboardInterrupt: 

In [ ]:
# print number of parameters in model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(encoder):,} trainable parameters')

# print model architecture
print(encoder)


### Masked Token Prediction

In [ ]:
def load_model_from_checkpoint(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
    model = EncoderModel(vocab_size=checkpoint['vocab_size'], 
                         embed_dim=checkpoint['embed_dim'], 
                         model_dim=checkpoint['model_dim'], 
                         n_layers=checkpoint['n_layers'], 
                         num_heads=checkpoint['num_heads'])
    model.load_state_dict(checkpoint['state_dict'])
    return model

def predict_masked_sent(model, tokenizer, text, top_k=5):
    '''
    Masked token inference. Credit: https://gist.github.com/yuchenlin/a2f42d3c4378ed7b83de65c7a2222eb2
    '''
    # Tokenize input
    text = "[CLS] %s [SEP]"%text
    tokenized_text = tokenizer.tokenize(text)
    masked_index = tokenized_text.index("[MASK]")
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    # tokens_tensor = tokens_tensor.to('cuda')    # if you have gpu

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    probs = torch.nn.functional.softmax(outputs, dim=-1)[0, masked_index]
    top_k_weights, top_k_indices = torch.topk(probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print("[MASK]: '%s'"%predicted_token, " | weights:", float(token_weight))

model = load_model_from_checkpoint('model_checkpoints/checkpoint.pth')
text = " I love [MASK] ."
predict_masked_sent(model, tokenizer=tokenizer, text=text, top_k=5)